In [9]:
from datasets import load_dataset
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
from gensim.models import FastText
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [10]:
# Загрузка токенизатора
nltk.download('punkt')

# Загрузка датасета AG News
dataset = load_dataset("ag_news")
train_texts = dataset["train"]["text"][:20000]  # Используем 20к примеров
test_texts = dataset["test"]["text"][:5000]
y_train = dataset["train"]["label"][:20000]
y_test = dataset["test"]["label"][:5000]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\R1sed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# Токенизация и подготовка данных в формате для FastText
def prepare_fasttext_data(texts):
    return [word_tokenize(text.lower()) for text in texts]

train_tokens = prepare_fasttext_data(train_texts)
test_tokens = prepare_fasttext_data(test_texts)

In [12]:
# Параметры модели
vector_size = 300    # Размерность векторов
window = 5           # Размер окна контекста
min_count = 2        # Минимальная частота слова
workers = 4          # Количество ядер CPU
epochs = 20          # Количество эпох
min_n = 3            # Минимальная длина n-граммы
max_n = 6            # Максимальная длина n-граммы

# Обучение FastText
ft_model = FastText(
    sentences=train_tokens,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    epochs=epochs,
    min_n=min_n,
    max_n=max_n,
    sg=1              # Алгоритм skip-gram (1) или CBOW (0)
)

# Сохранение модели
ft_model.save("fasttext_agnews.model")

In [14]:
def text_to_vector(tokens, model):
    vectors = []
    for word in tokens:
        # FastText может создавать вектор для любых слов (даже OOV)
        vectors.append(model.wv[word])
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Преобразование данных
X_train = np.array([text_to_vector(text, ft_model) for text in train_tokens])
X_test = np.array([text_to_vector(text, ft_model) for text in test_tokens])

In [15]:
# Обучение Logistic Regression
clf = LogisticRegression(max_iter=1000, n_jobs=-1)
clf.fit(X_train, y_train)

# Оценка
y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred, average='macro')
print(f"F1-score: {f1:.4f}")

# Проверка работы с OOV-словами
print("\nВектор для неизвестного слова 'blockchainxyz':")

F1-score: 0.8737

Вектор для неизвестного слова 'blockchainxyz':
[-3.82529274e-02  2.58661985e-01 -1.08449005e-01  4.01287936e-02
 -1.14541642e-01 -8.41330737e-02  1.54377893e-01  1.71183705e-01
 -7.39903972e-02  4.38373759e-02  8.86511728e-02  8.29486698e-02
  1.59600124e-01 -1.54286653e-01  4.00234386e-02  1.41114127e-02
  1.52176902e-01 -6.21485822e-02  2.57988781e-01 -1.19228484e-02
 -1.53156325e-01 -2.01592907e-01 -1.21295549e-01 -2.18276493e-02
  7.23198056e-02  4.92800847e-02 -1.51804881e-02  1.17122218e-01
 -1.11812316e-02 -2.93574542e-01  9.23379231e-03 -5.28966449e-02
 -7.33050704e-02  2.74827391e-01  5.37489802e-02  1.68604236e-02
  1.74357370e-01 -6.99208379e-02  6.47733286e-02  2.48058029e-02
 -1.83793932e-01 -4.49070111e-02 -7.80573022e-03 -9.14867222e-02
  5.62316030e-02  5.28095290e-02  2.44664416e-01  4.90491465e-03
 -1.62708804e-01 -1.65015891e-01 -2.92946206e-04  1.57396793e-01
 -2.42161706e-01 -3.24974172e-02  1.04109928e-01 -1.37859479e-01
  1.48747370e-01  6.27609